In [317]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from torch.utils.data import Dataset, DataLoader
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import AdamW
import torch
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import numpy as np
from tqdm import tqdm
from sklearn.metrics import recall_score, precision_score, f1_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, classification_report

In [318]:
PRETRAINED_MODELS = {
    'bert': 'bert-base-uncased',
    'roberta': 'roberta-base',
    'xlnet': 'xlnet-large-cased',
    'xlm': 'xlm-mlm-en-2048',
    'distilbert': 'distilbert-base-uncased',
    'albert':'albert-base-v2'
}

MODEL_CLASSES = {
    'bert': (BertForSequenceClassification, BertTokenizer, BertConfig),
    #'roberta': (RobertaForSequenceClassification, RobertaTokenizer, RobertaConfig),
    #'xlnet': (XLNetForSequenceClassification, XLNetTokenizer, XLNetConfig),
    #'xlm': (XLMForSequenceClassification, XLMTokenizer, XLMConfig),
    #'distilbert': (DistilBertForSequenceClassification, DistilBertTokenizer, DistilBertConfig),
    #'albert':(AlbertForSequenceClassification,AlbertTokenizer, AlbertConfig)
}

MODEL_TYPE = 'bert'
PRETRAINED_MODEL_NAME = PRETRAINED_MODELS[MODEL_TYPE]

model_class, tokenizer_class, config_class = MODEL_CLASSES[MODEL_TYPE]

TRUNCATION = True
LEARNING_RATE = 1e-5
BATCH_SIZE = 32
EPOCHS = 50
WEIGHT_DECAY = 0.01

LABELS = 'R2DiscussionType'

In [319]:
df = pd.read_csv('../datasets/dataset1.csv')
# Dropping topic because it has only one value. It might be usefull if we had more data
df = df.drop(columns=['Old Code Book', 'Memo', 'Chat0CREW1B', 'Chat0CREW1', 'Response Number', 'Message Time', 'CollapsR2DiscussionTypeInterpNothers', 'R2DiscussionTypeInterpNothers'])
df.columns = [''.join([word[0].upper() + word[1:] for word in col.split()]) for col in df.columns]
df.columns = [col.replace(' ', '') for col in df.columns]

In [320]:
print(df.shape)

(609, 25)


In [321]:
df.columns

Index(['Course', 'BookID', 'Topic', 'Bookclub', 'Pseudonym', 'Message',
       'IsAnswer', 'Page', 'R1DiscussionType', 'R2DiscussionType',
       'R1DialogicSpell', 'BinaryR1DialogicSpell', 'R1Uptake',
       'BinaryR1Uptake', 'R2DialogicSpell', 'BinaryR2DialogicSpell',
       'R2Uptake', 'BinaryR2Uptake', 'Pseudonym.1', 'Message.1', 'Bookclub.1',
       'R1Question', 'R2Question', 'R1Pivot', 'R2Pivot'],
      dtype='object')

If R1 has a value and R2 does not should we trust R1? Let us look at some of the examples to see if this is needed

In [322]:
# Identify the columns that start with R1 and R2 and have the same suffix
r1_columns = [col for col in df.columns if col.startswith('R1') or col.startswith('BinaryR1')]
r2_columns = [col for col in df.columns if col.startswith('R2') or col.startswith('BinaryR2')]

# Make sure that the suffixes match
matched_columns = [(r1, r2) for r1 in r1_columns for r2 in r2_columns if r1[2:] == r2[2:] or (r1[9:] == r2[9:] and r1.startswith('Binary') and r2.startswith('Binary'))]
matched_columns
#
## Iterate over these pairs of columns
for r1_col, r2_col in matched_columns:
    # Find rows where the R2 version has NaN and the R1 version has a value
    condition = (df[r2_col].isna() | df[r2_col].eq('')) & df[r1_col].notna()
    if condition.any():# and "Dialogic" not in r1_col:
        print(f'Column {r2_col} has NaN where {r1_col} has a value')
        print(df.loc[condition, [r1_col, r2_col]])
        if "Pivot" in r2_col or "Upta" in r2_col or "Question" in r2_col:
            df.loc[condition, r2_col] = df.loc[condition, r1_col]
            print(f'Filled {r2_col} with {r1_col}')

Column R2DialogicSpell has NaN where R1DialogicSpell has a value
    R1DialogicSpell R2DialogicSpell
240                             NaN
261           Begin             NaN
268                             NaN
273                             NaN
274                             NaN
275                             NaN
276                             NaN
295           Break             NaN
303           Begin             NaN
315           Begin             NaN
316                             NaN
317                             NaN
318                             NaN
319                             NaN
320                             NaN
321                             NaN
322                             NaN
325                             NaN
326             End             NaN
362           Break             NaN
396           Break             NaN
501               2             NaN
503               1             NaN
507               1             NaN
529               4             NaN

It seems to not be needed because all cases it is hard for me to tell which one is correct R1 or R2. Both answers seem valid so I will keep it as is.

In [323]:
df.drop(columns=r1_columns, inplace=True)

Below I remove duplicate columns and check if there are any values that are nan in the one I keep and not the other. In this case there were none

In [324]:
dot1_columns = [col for col in df.columns if col.endswith('.1')]
original_columns = [col.rstrip('.1') for col in dot1_columns if col.rstrip('.1') in df.columns]

# For each pair of columns, if one column has a value and the other does not, keep the value
for orig_col, dot1_col in zip(original_columns, dot1_columns):
    if (df[orig_col].isna() & df[dot1_col].notna()).any():
        print(f'Column {orig_col} has a value where {dot1_col} has NaN')

# Remove the .1 columns
df = df.drop(columns=dot1_columns)

There are 3 discussion type columns. The collapsed one was apparently used for visualisation according to the given paper so it is not needed. This leaves us with R2DiscussionTypeInterpNothers and R2DiscussionType. The first seems to use interpreatation instead of seminar and that is not in the codebook so we will use the first column. I will now look at the unique values of the columns and see their counts

In [325]:
counts = df['R2Uptake'].value_counts(dropna=False)
print(counts[0]/counts.sum())
counts

0.39080459770114945


/tmp/ipykernel_321966/412174125.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(counts[0]/counts.sum())


R2Uptake
NaN          238
Affirm       141
Elaborate    108
Filler        61
Clarify       50
Disagree      10
Prompt         1
Name: count, dtype: int64

Can't train on one sample so I make it NaN

In [326]:
df.loc[df["R2Uptake"] == "Prompt", "R2Uptake"] =  np.nan

We notice that there are not prompt or respond examples that can be found in the codebook. Also most are NaN which makes sense since maybe those are not uptakes. ALso NaNs are 40% of the data so we must achieve accuracy higher than 40 to have results better than just guessing NaN each time

In [328]:
df['Page'] = df['Page'].apply(str)
df['Page']

0      10
1      10
2      10
3      10
4      10
       ..
604     8
605    10
606    10
607    10
608    10
Name: Page, Length: 609, dtype: object

In [315]:
df['R2Question'].value_counts(dropna=False)

R2Question
NaN             526
C-LOT            44
C-HOT            23
O-HOT            11
O-LOT             8
O-HOT, C-LOT      1
Name: count, dtype: int64

Those who have repeating types can just be considered one type

In [283]:
df.loc[df['R2Question'] == "C-LOT, C-LOT", "R2Question"] = "C-LOT"
df.loc[df['R2Question'] == "C-HOT, C-HOT", "R2Question"] = "C-HOT"

O-COT does not exist in our Codebook so I looked at it and to me it looks like C-LOT since we have a probably a an answer in mind but we want clarification. This is also what R1 chose

In [284]:
df[df["R2Question"] == 'O-COT']["Message"]

241    Which questions? like the prompt?
Name: Message, dtype: object

In [285]:
df.loc[df['R2Question'] == 'O-COT', 'R2Question'] = 'C-LOT'

In [286]:
df[df["R2Question"] == 'O-HOT, C-LOT']["Message"]

360    Ok, so what happens then? Do we choose which d...
Name: Message, dtype: object

For the above sentence we need to divide into two samples

In [287]:
if LABELS == "R2Question":
    index = df.loc[df['Message'].str.startswith('Ok, so what happens then? Do we choose which door she opens?')].index[0]
    new_row = df.iloc[index].copy()
    new_row['Message'] =  "Do we choose which door she opens?"
    new_row['R2Question'] = 'C-LOT'
    new_row = pd.DataFrame(new_row).T
    df.at[index, 'Message'] = "Ok, so what happens then?"
    df.at[index, 'R2Question'] = 'O-HOT'
    df = pd.concat([df.iloc[:index + 1, ], new_row, df.iloc[index + 1:, ]], ignore_index=True) 


For the one with the '?' based on my and R1's judgment it is C-LOT. Since he askes a questions with an answer in mind.

Whole question:
"I feel the flow doesn't entirely match the original story, but I felt it was awkwardly written and okay. We can add (ending) before it?"

In [288]:
df[df["R2Question"] == 'C-LOT? C-HOT?']["Message"]

407    I feel the flow doesn't entirely match the ori...
Name: Message, dtype: object

In [289]:
df.loc[df["R2Question"] == 'C-LOT? C-HOT?', "R2Question"] = 'C-LOT'

We see that we have few samples compared to the total samples size that have a Question category. Also we have one sample with 2 labels. Which is likely not enough for our model to learn anything. This can probably be dropped when training and the ones that are C-LOT C-LOT can be converted just to a single C-LOT. Another options is to split the sentence somehow and classify separately and then combine the labels. This seems like a good idea and I might try it. For the C-LOT? C-HOT? it seems the expert is unsure what to write so we will either drop it or trust R1 which chose for that sample C-LOT.

In [290]:
df['R2DiscussionType'].value_counts(dropna=False)

R2DiscussionType
Seminar                  331
Deliberation              85
Social                    69
UX                        47
Procedure                 46
Imaginative entry         17
Other                      6
Seminar, Deliberation      2
Imaginative                2
Social, Deliberation       1
Deliberation, Seminar      1
Social, Procedure          1
Imaginative Entry          1
Name: count, dtype: int64

In [314]:
df['R2Pivot'].value_counts(dropna=False)

R2Pivot
NaN                                                           521
Seminar to Deliberation                                        17
Seminar to Social/Procedure/UX                                 14
Deliberation to Seminar                                        13
Social/Procedure/UX to Seminar                                  9
Deliberation to Social/Procedure/UX                             9
Social/Procedure/UX to Deliberation                             7
Social/Procedure/UX to Social/Procedure/UX                      7
Seminar to Social/Procedure/UX to Seminar                       4
Imaginative entry to Seminar                                    3
Seminar to Imaginative entry                                    3
Social/Procedure/UX to Imaginative                              1
Imaginative to Deliberation/Social/Procedure/UX                 1
Imaginative to Social/Procedure/UX                              1
Social/Procedure/UX to Deliberation to Social/Procedure/UX      1
Se

In [292]:
df.loc[df["R2Pivot"] == "Deliberationa to Seminar", "R2Pivot"] = "Deliberation to Seminar"
df.loc[df["R2Pivot"] == "Delibration to Seminar", "R2Pivot"] = "Deliberation to Seminar"
df.loc[df["R2Pivot"] == "Imaginative to Seminar", "R2Pivot"] = "Imaginative entry to Seminar"
df.loc[df["R2Pivot"] == "Seminar to Imaginative", "R2Pivot"] = "Seminar to Imaginative entry"

In [313]:
value_counts = df['R2Pivot'].value_counts(dropna=True)
total = value_counts.sum()
print(total)

92


For this one we don't have many samples separately but maybe we can make a binary classifier and just take the previous and next category. The classifier will just say pivot yes or no.

In [294]:
df['IsAnswer'].value_counts(dropna=False)

IsAnswer
No     501
NaN    106
         2
Name: count, dtype: int64

Is Answer is either No or NaN. The NaN value does not mean it is an answer. I looked at the fields that were labelled NaN and they can also be a greeting which is not an answer or a statement which is not always an answer. Because of this I believe this column is not useful for training and I will drop it

In [295]:
df.drop(columns=['IsAnswer'], inplace=True)

In [296]:
df['BookID'].value_counts(dropna=False)

BookID
260    421
261    188
Name: count, dtype: int64

In [297]:
df['Bookclub'].value_counts(dropna=False)

Bookclub
Book Club One      275
Book Club Four     192
Book Club Two       94
Book Club Three     30
Book Club Five      18
Name: count, dtype: int64

For BookID, Page and Bookclub I am unsure if they will be usefull in the end. My reasoning is because we want to generalise and what if we add a Bookclub 6 and if the model has never seen it. Or if we add a new book but the model has learned something about the specific books 260 and 261. Because of this I believe it is better to drop these columns. I was considering maybe they could be used for ids of different discussions since only a club or only a book is not enough to determine a discussion and we want to classify sentences in separate probably. If we want context we can try to use history instead of these ids.

In [298]:
df['PrevMessage'] = ''
for i in range(1, len(df)):
    if (df.loc[i, 'BookID'] == df.loc[i-1, 'BookID'] and
        df.loc[i, 'Bookclub'] == df.loc[i-1, 'Bookclub'] and
        df.loc[i, 'Course'] == df.loc[i-1, 'Course']):
        
        df.loc[i, 'PrevMessage'] = df.loc[i-1, 'Message']

In [299]:
prev_3 = []

prev_categories = [''] * len(df)

for i in range(1, len(df)):
    if df.iloc[i][['BookID', 'Bookclub', 'Course']].equals(df.iloc[i-1][['BookID', 'Bookclub', 'Course']]):
        if pd.isna(df.iloc[i-1][LABELS]):
            prev_3.append('')
            continue

        prev_3.append(df.iloc[i-1]['R2DiscussionType'] + '-' \
                        + df.iloc[i-1]['Pseudonym']  + "-" \
                        + df.iloc[i-1]["Page"] +  df.iloc[i-1]['R2Uptake'])
        
        if len(prev_3) > 3:
            prev_3.pop(0)
    else:
        prev_3 = []
    prev_categories[i] = '|'.join(prev_3)

df['PrevCategories'] = prev_categories

In [300]:
df.drop(columns=['BookID', 'Bookclub', 'Page', 'Course', 'Topic'], inplace=True)

In [301]:
# Select all rows where 'R2Pivot' is not NaN
non_nan_rows = df[df['R2Pivot'].notna()]

# Get the indices of the non-NaN rows
indices = non_nan_rows.index

# Print the current 'R2DiscussionType' for these rows and the 'R2DiscussionType' from the previous row in the original DataFrame
res = zip(non_nan_rows['R2DiscussionType'], df.loc[indices - 1, 'R2DiscussionType'] if indices[0] > 0 else None)

counter = 0
for idx, (next, prev) in enumerate(res):
    if prev in non_nan_rows.iloc[idx]['R2Pivot'] and next in non_nan_rows.iloc[idx]['R2Pivot']:
        counter += 1
    else:
        print(non_nan_rows.iloc[idx]['Message'])
        print(non_nan_rows.iloc[idx]['R2Pivot'])
        print(prev, next)

print(counter/len(non_nan_rows))

So, where have we landed: Lady or tiger? Trial or coverup? I think we just have to write something in the space below.
Seminar to Deliberation
Seminar Seminar, Deliberation
I agree that the princess could just banish the new couple away too. Do we have to agree on a single ending?
Seminar to Deliberation
Seminar Seminar, Deliberation
I don't think we have to agree on a single ending as both are viable. I still will go with the Lady as being behind the curtain. The King will be upset with this but will forgive his daughter. He may do something else to the boyfriend later.
Deliberation to Seminar
Seminar, Deliberation Seminar
Ok, I had to reset the book to make this work!
Social/Procedure/UX to Social/Procedure/UX
Deliberation Deliberation
Yay you got it!
Social/Procedure/UX to Social/Procedure/UX
Deliberation Social, Deliberation
Live without was the first one I put. Then Trust, Love, and then Know
Social/Procedure/UX to Deliberation
Social, Deliberation Deliberation
Oh I see, there are

In [302]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [303]:
df['R2DiscussionType'].value_counts()

R2DiscussionType
Seminar                  331
Deliberation              85
Social                    69
UX                        47
Procedure                 46
Imaginative entry         17
Other                      6
Seminar, Deliberation      2
Imaginative                2
Social, Deliberation       1
Deliberation, Seminar      1
Social, Procedure          1
Imaginative Entry          1
Name: count, dtype: int64

In [304]:
if LABELS == "R2DiscussionType":
    df.loc[df['R2DiscussionType'] == 'Imaginative Entry', 'R2DiscussionType'] = 'Imaginative entry'
    df.loc[df['R2DiscussionType'] == 'Imaginative', 'R2DiscussionType'] = 'Imaginative entry'
    df.loc[df['R2DiscussionType'] == 'Deliberation, Seminar', 'R2DiscussionType'] = 'Seminar, Deliberation'

In [305]:
df[df['R2DiscussionType'] == 'Social, Deliberation']['Message']

237    Yay you got it!
Name: Message, dtype: object

The above example seems like only Social to me. It can also be procedure due to you got it. Since it is only one sample and I do not believe it will effect our results I will just mark it as social.

In [306]:
if LABELS == "R2DiscussionType":
    df.loc[df['R2DiscussionType'] == 'Social, Deliberation', 'R2DiscussionType'] = 'Social'

In [307]:
df[df['R2DiscussionType'] == 'Seminar, Deliberation']['Message']

202    So, where have we landed: Lady or tiger? Trial...
226    I agree that the princess could just banish th...
245    Oh I see, there are two questions here. I feel...
Name: Message, dtype: object

Full versions: \
1: So, where have we landed: Lady or tiger? Trial or coverup? I think we just have to write something in the space below. \
2: I agree that the princess could just banish the new couple away too. Do we have to agree on a single ending? \
3: Oh I see, there are two questions here. I feel that the princess would have led him to the tiger 

These are combination of two sentences in a single sample that each are from a different category. I will divide the sample into two and mark each with the corresponding category.

In [308]:
if LABELS == "R2DiscussionType":
    index = df.loc[df['Message'].str.startswith('Oh I see, there are two questions here')].index[0]
    new_row = df.iloc[index].copy()
    new_row['Message'] = "I feel that the princess would have led him to the tiger"
    new_row['R2DiscussionType'] = 'Seminar'
    new_row = pd.DataFrame(new_row).T
    df.at[index, 'Message'] = "Oh I see, there are two questions here."
    df.at[index, 'R2DiscussionType'] = 'Deliberation'
    df = pd.concat([df.iloc[:index + 1, ], new_row, df.iloc[index + 1:, ]], ignore_index=True) 

    index = df.loc[df['Message'].str.startswith('I agree that the princess could just banish the new couple away too.')].index[0]
    new_row = df.iloc[index].copy()
    new_row['Message'] = "Do we have to agree on a single ending?"
    new_row['R2DiscussionType'] = 'Deliberation'
    new_row = pd.DataFrame(new_row).T
    df.at[index, 'Message'] = "I agree that the princess could just banish the new couple away too."
    df.at[index, 'R2DiscussionType'] = 'Seminar'
    df = pd.concat([df.iloc[:index + 1, ], new_row, df.iloc[index + 1:, ]], ignore_index=True) 

    index = df.loc[df['Message'].str.startswith('So, where have we landed: Lady or tiger? Trial or coverup?')].index[0]
    new_row = df.iloc[index].copy()
    new_row['Message'] = "I think we just have to write something in the space below."
    new_row['R2DiscussionType'] = 'Deliberation'
    new_row = pd.DataFrame(new_row).T
    df.at[index, 'Message'] = "So, where have we landed: Lady or tiger? Trial or coverup?"
    df.at[index, 'R2DiscussionType'] = 'Seminar'
    df = pd.concat([df.iloc[:index + 1, ], new_row, df.iloc[index + 1:, ]], ignore_index=True) 

In [309]:
df[df['R2DiscussionType'] == 'Social, Procedure']['Message']

315    Looks good! I guess we can complete the post s...
Name: Message, dtype: object

Here Looks good is the social part and the rest the Procedure so I will divide again

In [310]:
if LABELS == "R2DiscussionType":
    index = df.loc[df['Message'].str.startswith('Looks good! I guess we can complete the post survey')].index[0]
    new_row = df.iloc[index].copy()
    new_row['Message'] = "I guess we can complete the post survey and submit our assignment"
    new_row['R2DiscussionType'] = 'Procedure'
    new_row = pd.DataFrame(new_row).T
    df.at[index, 'Message'] = "Looks good!"
    df.at[index, 'R2DiscussionType'] = 'Social'
    df = pd.concat([df.iloc[:index + 1, ], new_row, df.iloc[index + 1:, ]], ignore_index=True) 

In [312]:
df.value_counts('R2DiscussionType')

R2DiscussionType
Seminar              334
Deliberation          88
Social                71
Procedure             47
UX                    47
Imaginative entry     20
Other                  6
Name: count, dtype: int64

In [172]:
class R2UptakeDataset(Dataset):
    def __init__(self, X, y, tokenizer, max_length, enc):
        self.X = X.values
        self.tokenizer = tokenizer
        self.max_length = max_length
        # Fit the label binarizer and transform the labels into one-hot encoded format
        self.labels = enc.transform(y.values.reshape(-1, 1)).toarray()

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        # Encode the utterance using the provided tokenizer
        encoding = self.tokenizer.encode_plus(
            self.X[idx],
            add_special_tokens=True,
            max_length = self.max_length,
            return_token_type_ids=True,
            padding='max_length',
            return_attention_mask=True,
            truncation=TRUNCATION,
            return_tensors='pt'
        )
        # Convert the list of strings into a one-hot encoded format
        label = self.labels[idx]  # This should now be a binary vector instead of a list of strings

        # Return the encoding and the label
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.float),
            'token_type_ids': encoding['token_type_ids'].flatten(),
        }

In [173]:
r2columns = [col for col in df.columns if 'R2' in col]
X = df.drop(columns=r2columns).fillna('')
y = df[LABELS].fillna('None')

X["texts"] = X['Message'] + "|" + X['PrevCategories'] + "|" + X['Pseudonym'] + "-" + X['Page']

if LABELS == 'R2Uptake':
    X["texts"] = X["texts"] + "-" + X['R2DiscussionType']


tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

In [174]:
y.value_counts()

R2DiscussionType
Seminar              334
Deliberation          88
Social                71
Procedure             47
UX                    47
Imaginative entry     20
Other                  6
Name: count, dtype: int64

In [175]:
num_labels = len(df[LABELS].unique())
num_labels

7

In [176]:
X_train_tmp, X_test, y_train_tmp, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train_tmp, y_train_tmp, test_size=0.3, random_state=42, stratify=y_train_tmp)

enc = OneHotEncoder()
enc.fit(y_train.values.reshape(-1, 1))

OneHotEncoder()

In [178]:
train_texts = X_train['texts']
train_labels = y_train

val_texts = X_val['texts']
val_labels = y_val

test_texts = X_test['texts']
test_labels = y_test

longest_train_data = train_texts[train_texts.str.len().idxmax()]
max_length = min(2 ** (len(tokenizer.tokenize(longest_train_data))-1).bit_length(), 512)
print(train_labels.value_counts())

train_data = R2UptakeDataset(train_texts, train_labels, tokenizer,max_length, enc)
val_data = R2UptakeDataset(val_texts, val_labels, tokenizer, max_length, enc)
test_data = R2UptakeDataset(test_texts, test_labels, tokenizer, max_length, enc)

le = LabelEncoder()
train_labels_encoded = le.fit_transform(train_labels)
val_labels_encoded = le.transform(val_labels)

train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)

R2DiscussionType
Seminar              187
Deliberation          49
Social                40
UX                    27
Procedure             26
Imaginative entry     11
Other                  3
Name: count, dtype: int64


In [179]:
y_train.value_counts()

R2DiscussionType
Seminar              187
Deliberation          49
Social                40
UX                    27
Procedure             26
Imaginative entry     11
Other                  3
Name: count, dtype: int64

In [180]:
train_texts.iloc[1]

'I think the tiger was behind the door. Mostly because of the princess being described as "hot-blooded and semi-barbaric."|Seminar|Seminar|Seminar'

In [181]:
train_labels.iloc[3]

'Deliberation'

In [182]:
#HIDDEN_WEIGHTS = 768
HIDDEN_WEIGHTS = 256
DROPOUT = 0.3
class BERTClass(torch.nn.Module):
    def __init__(self, pretrained_model_name, num_labels):
        super(BERTClass, self).__init__()
        self.num_labels = num_labels
        self.l1 = model_class.from_pretrained(pretrained_model_name, num_labels=self.num_labels)
        self.pre_classifier = torch.nn.Linear(self.num_labels, HIDDEN_WEIGHTS)
        self.dropout = torch.nn.Dropout(DROPOUT)
        self.classifier = torch.nn.Linear(HIDDEN_WEIGHTS, self.num_labels)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        output = output.view(-1, self.num_labels)  # Reshape the output
        return output

model = BERTClass(PRETRAINED_MODEL_NAME, num_labels)
model = model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [183]:
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)

weights = torch.tensor(class_weights, dtype=torch.float)

if torch.cuda.is_available():
    weights = weights.to('cuda')
    
criterion = torch.nn.CrossEntropyLoss(weight=weights)

def loss_fn(outputs, targets):
    return criterion(outputs, targets)

optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

In [184]:
def valid(model, valid_dataloader):
    val_targets = []
    val_outputs = []
    
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in tqdm(valid_dataloader):
            input_ids = batch['input_ids'].to(device, dtype=torch.long)
            attention_mask = batch['attention_mask'].to(device, dtype=torch.long)
            token_type_ids = batch['token_type_ids'].to(device, dtype=torch.long)
            labels = batch['labels'].to(device, dtype=torch.float)

            outputs = model(input_ids, attention_mask, token_type_ids)

            loss = loss_fn(outputs, labels)
            val_loss += loss.item()

            val_targets.extend(labels.cpu().detach().numpy().tolist())
            val_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
            

    val_loss /= len(valid_dataloader)
    
    return val_loss, val_targets, val_outputs

In [185]:
def load_ckp(checkpoint_fpath, model, enc):
    checkpoint = torch.load(checkpoint_fpath, map_location=device)
    model.load_state_dict(checkpoint['state_dict'])
    enc.set_params(**checkpoint['enc'])
    return model, enc

def save_ckp(state, best_model_path):
    torch.save(state, best_model_path)

In [186]:
def train(model, train_dataloader):
    model.train()
    train_loss = 0
    for batch in tqdm(train_dataloader):
        input_ids = batch['input_ids'].to(device, dtype=torch.long)
        attention_mask = batch['attention_mask'].to(device, dtype=torch.long)
        token_type_ids = batch['token_type_ids'].to(device, dtype=torch.long)
        labels = batch['labels'].to(device, dtype=torch.float)

        model.zero_grad()
        outputs = model(input_ids, attention_mask, token_type_ids)

        loss = loss_fn(outputs, labels)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()

    train_loss /= len(train_dataloader)
    
    return train_loss

In [187]:
def train_model(num_epochs, train_dataloader, valid_dataloader, model, optimizer, best_model_path = "./", patience=1):
    valid_loss_min = np.Inf

    num_not_improved = 0
    for epoch in range(1, num_epochs):
        print()
        print("#################### Epoch {}: Training Start    ####################".format(epoch))
        train_loss = train(model, train_dataloader)
        print('#################### Epoch {}: Training End      ####################'.format(epoch))

        print()
        print("#################### Epoch {}: Validation Start ####################".format(epoch))

        valid_loss, val_targets, val_outputs = valid(model, valid_dataloader)
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch, train_loss, valid_loss))

        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min, valid_loss))

            checkpoint = {
                        'state_dict': model.state_dict(),
                        'enc' : enc.get_params()
                    }

            save_ckp(checkpoint, best_model_path)
            valid_loss_min = valid_loss
            num_not_improved = 0
        else:
            num_not_improved += 1
            if num_not_improved >= patience:
                print('Not improvement for more than:', num_not_improved)
                break
            
        print("#################### Epoch {}: Validation End   ####################".format(epoch))
        print()

    print("#################### Training finished     ####################")
    return model

In [188]:
print('No saved model found. Need to be train from scratch.')
trained_model = train_model(EPOCHS, train_loader, val_loader, model, optimizer)

No saved model found. Need to be train from scratch.

#################### Epoch 1: Training Start    ####################


  0%|          | 0/11 [00:16<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
test_loss, test_labels , test_predictions_probs = valid(trained_model, test_loader)

100%|██████████| 4/4 [00:45<00:00, 11.41s/it]


In [ ]:
test_loss

0.4385611116886139

In [ ]:
test_predictions = [ prob_list == np.max(prob_list) for prob_list in test_predictions_probs ]

In [ ]:
test_predictions

[[False, False, False, False, False, False],
 [False, False, False, False, False, False],
 [False, False, False, False, False, False],
 [False, False, False, False, False, False],
 [False, False, False, False, False, False],
 [False, False, False, False, False, False],
 [False, False, False, False, False, False],
 [False, False, False, False, False, False],
 [False, False, False, False, False, False],
 [False, False, False, False, False, False],
 [False, False, False, False, False, False],
 [False, False, False, False, False, False],
 [False, False, False, False, False, False],
 [False, False, False, False, False, False],
 [False, False, False, False, False, False],
 [False, False, False, False, False, False],
 [False, False, False, False, False, False],
 [False, False, False, False, False, False],
 [False, False, False, False, False, False],
 [False, False, False, False, False, False],
 [False, False, False, False, False, False],
 [False, False, False, False, False, False],
 [False, F

In [ ]:
print('Accuracy:', accuracy_score(test_labels, test_predictions))
print('Precision:', precision_score(test_labels, test_predictions, average='weighted'))
print('Recall:', recall_score(test_labels, test_predictions, average='weighted'))
print('F1:', f1_score(test_labels, test_predictions, average='weighted'))

report = classification_report(test_labels, test_predictions, target_names=enc.categories_[0])
print(report)

Accuracy: 0.0
Precision: 0.0
Recall: 0.0
F1: 0.0
              precision    recall  f1-score   support

      Affirm       0.00      0.00      0.00        28
     Clarify       0.00      0.00      0.00        10
    Disagree       0.00      0.00      0.00         2
   Elaborate       0.00      0.00      0.00        22
      Filler       0.00      0.00      0.00        12
        None       0.00      0.00      0.00        48

   micro avg       0.00      0.00      0.00       122
   macro avg       0.00      0.00      0.00       122
weighted avg       0.00      0.00      0.00       122
 samples avg       0.00      0.00      0.00       122



/home/nikolay/EMAI/Ljublajna/NLP/ul-fri-nlp-course-project-processingbit/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/nikolay/EMAI/Ljublajna/NLP/ul-fri-nlp-course-project-processingbit/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/nikolay/EMAI/Ljublajna/NLP/ul-fri-nlp-course-project-processingbit/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no p

IDEAS: Try to give the models also the users when passing previous categories. In the case of uptake it might be usefull? Cause we will be uptaking from him if it is only us speaking it is less likely to have an uptake.